# Initial conflicting factor:  To determine the pocket quality. 
    - pair-wise distance within the pocket 
# Dynamics factor: 
    - Performance in ligand pair-wise distance to pocket 


In [2]:
import pytraj as pt 
import numpy as np 

class label_gen: 
  def __init__(self, dic):
    print("initialize")
    

    

a = label_gen()

TypeError: __init__() missing 1 required positional argument: 'dic'

In [262]:
import pytraj as pt 
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt 

a = "/home/miemie/storage/WDRed/downloads/N_008_LIG_END.pdb"
def conflictfactor(pdbfile, ligname, cutoff=5): 
  VDWRADII = {'1': 1.1, '2': 1.4, '3': 1.82, '4': 1.53, '5': 1.92, '6': 1.7, '7': 1.55, '8': 1.52, 
    '9': 1.47, '10': 1.54, '11': 2.27, '12': 1.73, '13': 1.84, '14': 2.1, '15': 1.8, 
    '16': 1.8, '17': 1.75, '18': 1.88, '19': 2.75, '20': 2.31, '28': 1.63, '29': 1.4, 
    '30': 1.39, '31': 1.87, '32': 2.11, '34': 1.9, '35': 1.85, '46': 1.63, '47': 1.72, 
    '48': 1.58, '50': 2.17, '51': 2.06, '53': 1.98, '54': 2.16, '55': 3.43, '56': 2.68, 
    '78': 1.75, '79': 1.66, '82': 2.02, '83': 2.07
  }
  traj = pt.load(pdbfile, top=pdbfile); 
  traj.top.set_reference(traj[0]); 
  pocket_atoms = traj.top.select(f":{ligname}<:{cutoff}"); 
  atoms = np.array([*traj.top.atoms])[pocket_atoms]; 
  coords = traj.xyz[0][pocket_atoms]; 
  atomnr = len(pocket_atoms); 
  cclash=0; 
  ccontact = 0;
  for i, coord in enumerate(coords):
    partners = [atoms[i].index]
    for j in list(atoms[i].bonded_indices()):
      if j in pocket_atoms:
        partners.append(j)
    partners.sort()
    otheratoms = np.setdiff1d(pocket_atoms, partners)
    ret = distance_matrix([coord], traj.xyz[0][otheratoms])
    thisatom = atoms[i].atomic_number
    vdw_pairs = np.array([VDWRADII[str(i.atomic_number)] for i in np.array([*traj.top.atoms])[otheratoms]]) + VDWRADII[str(thisatom)]
    cclash += np.count_nonzero(ret < vdw_pairs - 1.25)
    ccontact += np.count_nonzero(ret < vdw_pairs + 0.4)
    
    st = (ret < vdw_pairs - 1.25)[0]; 
    if np.count_nonzero(st) > 0:
      partatoms = np.array([*traj.top.atoms])[otheratoms][st]; 
      thisatom = np.array([*traj.top.atoms])[atoms[i].index]; 
      for part in partatoms:
        dist = distance_matrix([traj.xyz[0][part.index]], [traj.xyz[0][thisatom.index]]); 
        print(f"Found clash between: {thisatom.name}({thisatom.index}) and {part.name}({part.index}); Distance: {dist.squeeze().round(3)}")
    
  factor = 1 - ((cclash/2)/((ccontact/2)/atomnr))
  print(f"Clashing factor: {round(factor,3)}; Atom selected: {atomnr}; Contact number: {ccontact}; Clash number: {cclash}"); 
  return factor
conflictfactor(a,"LIG", cutoff=6)

Clashing factor: 1.0; Atom selected: 440; Contact number: 3810; Clash number: 0


1.0

In [27]:
VDWRADII = {'AA': 1.85, 'AG': 1.72, 'AL': 1.84, 'AR': 1.88, 'AT': 2.02, 'AU': 1.66, 'B': 1.92, 'BA': 2.68,
  'BE': 1.53, 'BI': 2.07, 'BR': 1.85, 'C': 1.7, 'CA': 2.31, 'CD': 1.58, 'CL': 1.75, 'Cl': 1.75,
  'CS': 3.43, 'CU': 1.4, 'F': 1.47, 'FR': 3.48, 'GA': 1.87, 'GE': 2.11, 'H': 1.1,
  'HE': 1.4, 'HH': 1.55, 'I': 1.98, 'IN': 1.93, 'K': 2.75, 'KR': 2.02, 'LI': 1.82, 'MG': 1.73,
  'N': 1.55, 'NA': 2.27, 'NE': 1.54, 'NI': 1.63, 'O': 1.52, 'P': 1.8, 'PB': 2.02, 'PD': 1.63,
  'PO': 1.97, 'PT': 1.75, 'RA': 2.83, 'RN': 2.2, 'RR': 3.03, 'S': 1.8, 'SB': 2.06, 'SE': 1.9,
  'SI': 2.1, 'SN': 2.17, 'SR': 2.49, 'TE': 2.06, 'TL': 1.96, 'U': 1.86, 'XE': 2.16, 'ZN': 1.39,
}

print(VDWRADII["C"],
VDWRADII["H"],
VDWRADII["N"],
VDWRADII["O"])

np.mean(list(VDWRADII.values()))


np.mean([1.7,1.1,1.55,1.52])

1.7 1.1 1.55 1.52


1.4674999999999998

In [208]:
VDWRADII = {
    '47': 1.72, '13': 1.84, '18': 1.88, '79': 1.66, '5': 1.92, '56': 2.68,
    '4': 1.53, '83': 2.07, '35': 1.85, '6': 1.7, '20': 2.31, '48': 1.58, '17': 1.75,
    '55': 3.43, '29': 1.4, '9': 1.47, '31': 1.87, '32': 2.11, '1': 1.1,
    '2': 1.4, '53': 1.98, '19': 2.75, '3': 1.82, '12': 1.73,
    '7': 1.55, '11': 2.27, '10': 1.54, '28': 1.63, '8': 1.52, '15': 1.8, '82': 2.02, '46': 1.63,
    '78': 1.75, '16': 1.8, '51': 2.06, '34': 1.9, '14': 2.1, '50': 2.17, '54': 2.16, '30': 1.39,
  }
print(VDWRADII.keys())
print(len(VDWRADII.keys()))
x={}
for i in range(1, 100):
  if str(i) in VDWRADII.keys():
    x[str(i)] = VDWRADII[str(i)]
    print(VDWRADII[str(i)])
  
print(x)

dict_keys(['47', '13', '18', '79', '5', '56', '4', '83', '35', '6', '20', '48', '17', '55', '29', '9', '31', '32', '1', '2', '53', '19', '3', '12', '7', '11', '10', '28', '8', '15', '82', '46', '78', '16', '51', '34', '14', '50', '54', '30'])
40
1.1
1.4
1.82
1.53
1.92
1.7
1.55
1.52
1.47
1.54
2.27
1.73
1.84
2.1
1.8
1.8
1.75
1.88
2.75
2.31
1.63
1.4
1.39
1.87
2.11
1.9
1.85
1.63
1.72
1.58
2.17
2.06
1.98
2.16
3.43
2.68
1.75
1.66
2.02
2.07
{'1': 1.1, '2': 1.4, '3': 1.82, '4': 1.53, '5': 1.92, '6': 1.7, '7': 1.55, '8': 1.52, '9': 1.47, '10': 1.54, '11': 2.27, '12': 1.73, '13': 1.84, '14': 2.1, '15': 1.8, '16': 1.8, '17': 1.75, '18': 1.88, '19': 2.75, '20': 2.31, '28': 1.63, '29': 1.4, '30': 1.39, '31': 1.87, '32': 2.11, '34': 1.9, '35': 1.85, '46': 1.63, '47': 1.72, '48': 1.58, '50': 2.17, '51': 2.06, '53': 1.98, '54': 2.16, '55': 3.43, '56': 2.68, '78': 1.75, '79': 1.66, '82': 2.02, '83': 2.07}


In [259]:
1-3/((24974/2)/2400)

0.4234003363498038